# RMSprop

### 개념 요약
RMSprop(Root Mean Square Propagation)은 각 파라미터에 맞는 적응적 학습률(adaptive learning rate)을 사용하는 최적화 알고리즘임. 이 알고리즘은 경사(gradient)의 크기에 따라 학습률을 조절하여, 경사가 가파른 곳에서는 보폭을 줄이고 완만한 곳에서는 보폭을 늘려 효율적인 학습을 가능하게 함.

특히 Adagrad가 학습을 오래 진행할 경우 학습률이 0에 가까워져 더 이상 학습되지 않는 문제를 해결하기 위해 제안되었음. RMSprop은 단순히 경사의 제곱을 계속 더하는 대신, 지수 이동 평균(exponential moving average)을 사용하여 최근 경사의 정보에 더 큰 가중치를 둠.

### 📐 수학적 배경
RMSprop의 파라미터 업데이트 과정은 다음과 같음.

1. **경사 제곱의 지수 이동 평균(Exponential Moving Average of Squared Gradients) 업데이트:**
$$ E[g^2]_t = \gamma E[g^2]_{t-1} + (1 - \gamma) g_t^2 $$

2. **파라미터(Weight) 업데이트:**
$$ W_{t+1} = W_t - \frac{\eta}{\sqrt{E[g^2]_t + \epsilon}} g_t $$

* $W$: 업데이트할 파라미터
* $g_t$: 현재 타임스텝 $t$에서의 경사(gradient)
* $E[g^2]_t$: 경사 제곱($g^2$)의 지수 이동 평균. 최근 경사들의 크기를 나타내는 지표
* $\gamma$: 지수 이동 평균을 위한 감쇠율(decay rate). 보통 0.9와 같은 값을 사용
* $\eta$: 초기 학습률(learning rate)
* $\epsilon$: 분모가 0이 되는 것을 방지하기 위한 작은 상수 (예: $10^{-8}$)

### 라이브러리 임포트 (Setup)

In [ ]:
# !pip install numpy matplotlib

import numpy as np
import matplotlib.pyplot as plt

### 예제 데이터 준비 (Data Preparation)
RMSprop의 적응적 학습률 기능을 확인하기 위해, 각 차원별로 곡률이 다른 손실 함수 $f(x, y) = 0.1x^2 + 5y^2$를 사용함. 이 함수의 경사는 $\nabla f = [0.2x, 10y]$임. x축 방향은 완만하고 y축 방향은 가파르기 때문에, 최적화 알고리즘이 각 방향에 맞춰 보폭을 다르게 조절해야 효율적임.

In [ ]:
# 손실 함수 정의
def loss_function(w):
    return 0.1 * w[0]**2 + 5 * w[1]**2

# 손실 함수의 경사(미분) 정의
def gradient(w):
    return np.array([0.2 * w[0], 10 * w[1]])

### 핵심 로직 구현 (Code Implementation)
RMSprop 알고리즘을 직접 구현하여 파라미터가 각 차원에 따라 어떻게 다르게 업데이트되는지 확인함.

In [ ]:
# 하이퍼파라미터 설정
learning_rate = 0.5   # 초기 학습률
gamma = 0.9           # 감쇠율
epsilon = 1e-8        # 0으로 나누기 방지
n_iterations = 50     # 반복 횟수

# 파라미터 초기화
w = np.array([-10.0, 1.5])
E_g_sq = np.zeros_like(w) # 경사 제곱의 지수 이동 평균 (초기값은 0)
w_history = [w.copy()]

print("--- RMSprop 최적화 과정 ---")
for i in range(n_iterations):
    # 1. 현재 위치에서 경사 계산
    grad = gradient(w)
    
    # 2. 경사 제곱의 지수 이동 평균 업데이트
    E_g_sq = gamma * E_g_sq + (1 - gamma) * grad**2
    
    # 3. RMSprop을 이용한 파라미터 업데이트
    # 경사가 완만한 x축(grad[0])에 대해서는 E_g_sq[0]가 작아져 업데이트 폭이 커지고,
    # 경사가 가파른 y축(grad[1])에 대해서는 E_g_sq[1]가 커져 업데이트 폭이 작아짐.
    w = w - (learning_rate / np.sqrt(E_g_sq + epsilon)) * grad
    w_history.append(w.copy())
    
    if i % 5 == 0:
        print(f"Iter {i+1:2d}: Grad=[{grad[0]:6.3f}, {grad[1]:6.3f}], E_g_sq=[{E_g_sq[0]:6.3f}, {E_g_sq[1]:6.3f}], W=[{w[0]:6.3f}, {w[1]:6.3f}]")

### 📊 결과 확인 및 시각화 (Results & Visualization)
최적화 과정에서 파라미터가 이동한 경로를 등고선(contour) 플롯 위에 시각화함. x축(완만한 경사) 방향으로는 넓은 보폭으로, y축(가파른 경사) 방향으로는 좁은 보폭으로 움직이며 효율적으로 최저점을 찾아가는 것을 확인할 수 있음.

In [ ]:
w_history = np.array(w_history)

# 등고선 플롯을 위한 그리드 생성
x_mesh, y_mesh = np.meshgrid(np.linspace(-12, 12, 100), np.linspace(-2, 2, 100))
z_mesh = loss_function(np.array([x_mesh, y_mesh]))

plt.figure(figsize=(10, 7))
plt.contourf(x_mesh, y_mesh, z_mesh, levels=20, cmap='viridis_r', alpha=0.7)
plt.colorbar(label='Loss')

# 파라미터 업데이트 경로 시각화
plt.plot(w_history[:, 0], w_history[:, 1], 'o-', color='red', label='RMSprop Path')

plt.title('RMSprop Optimization Path', fontsize=16)
plt.xlabel('Parameter w_1')
plt.ylabel('Parameter w_2')
plt.legend()
plt.grid(True)
plt.axis('equal')
plt.show()

### 고려사항 (Considerations)

* <b>초기 학습률($\eta$)과 감쇠율($\gamma$)</b>: RMSprop의 성능은 `learning_rate`와 `gamma` 값에 민감할 수 있음. 일반적으로 `gamma`는 0.9로 설정하지만, 문제에 따라 조절이 필요할 수 있음.
* **Adam과의 관계**: Adam(Adaptive Moment Estimation) 최적화 기법은 RMSprop에 모멘텀(Momentum)을 결합한 방식임. Adam은 각 파라미터의 적응적 학습률과 함께 관성을 함께 고려하여 현재 가장 널리 사용되는 최적화 알고리즘 중 하나임.

### 최종 요약 (Conclusion)
RMSprop은 최근 경사들의 제곱 평균 제곱근(Root Mean Square)을 사용하여 각 파라미터의 학습률을 적응적으로 조절하는 알고리즘임. 이를 통해 손실 함수의 형태가 방향에 따라 크게 다른 경우에도 안정적이고 빠르게 최적점에 수렴할 수 있음.

예제에서 확인했듯이, RMSprop은 경사가 완만한 축으로는 빠르게 이동하고 가파른 축으로는 조심스럽게 이동하여, 비등방성(anisotropic)을 가진 함수 최적화에 매우 효과적임. 이는 Adagrad의 학습률이 단조롭게 감소하는 문제를 해결한 중요한 진전으로 평가받음.